In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
lst = list(range(0,7471,5))
#lst = list(range(0,100,5))

column_names = ['ReviewDate', 'ReviewerName', 'Rating', 'TextReview', 'StayDate', 'HelpfulVotes', 'ManagerResponseText', 'ManagerResponseDate']
df_reviews = pd.DataFrame(columns = column_names)

for i in lst:
    #access webpage
    if i != 0:
        url = 'https://www.tripadvisor.co.uk/Hotel_Review-g1480252-d316623-Reviews-or' + str(i) + '-Kuredu_Island_Resort_Spa-Kuredu.html#REVIEWS'
    else:
        url = 'https://www.tripadvisor.co.uk/Hotel_Review-g1480252-d316623-Reviews-Kuredu_Island_Resort_Spa-Kuredu.html'

    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    #user name
    mem_name = []
    mem = soup.find_all(class_='social-member-event-MemberEventOnObjectBlock__member--35-jC')
    for m in mem:
        try:
            mem_name.append(m.get_text())
        except AttributeError:
            mem_name.append('Nan')

    #user hometown
    #mem_home = []
    #mem_h = soup.find_all(class_='social-member-common-MemberHometown__hometown--3kM9S')
    #for m in mem_h:
      #  try:
      #      mem_home.append(m.get_text())
      #  except AttributeError:
       #     mem_home.append('Nan')


    #date of review
    date_r = []
    date_rev = soup.find_all(class_='social-member-event-MemberEventOnObjectBlock__event_type--3njyv')
    for d in date_rev:
        try:
            d = d.get_text()
            d = " ".join(d.split(' ')[-2:])
            date_r.append(d)
        except AttributeError:
            date_r.append('Nan')
            
    all_review = soup.find_all(class_='location-review-review-list-parts-SingleReview__mainCol--1hApa')
    
    #extract rating
    rating = []
    for rev in all_review:
        try:
            rate = rev.find_all(class_='ui_bubble_rating')
            rating.append(float((str(rate[-1]).split(' ')[-1].split('"')[0].split('_')[-1]))/10)
        except Exception:
            rating.append('Nan')

    #extract full text review
    full_text_rev = []
    for rev in all_review:
        try:
            r = rev.find_all(class_ ='location-review-review-list-parts-ExpandableReview__reviewText--gOmRC')
            for i in r:
                full_text_rev.append(i.get_text())
        except Exception:
            full_text_rev.append('Nan')

    #stay date
    stay_date = []
    for rev in all_review:
        try:
            dat = rev.find_all(class_='location-review-review-list-parts-EventDate__event_date--1epHa')
            stay_date.append(str(dat[0]).split('</span>')[-2].strip())
        except Exception:
            stay_date.append('Nan')

    #trip type if given
    #trip_type = []
    #for rev in all_review:
     #   try:
      #      trip_type.append(rev.find(class_='location-review-review-list-parts-TripType__trip_type--3w17i').get_text())
       # except AttributeError:
        #    trip_type.append('Nan')

    #room tip
    #room_tip = []
    #for rev in all_review:
     #   try:
      #      room_tip.append(rev.find(class_='location-review-review-list-parts-InlineRoomTip__tipline--1NsZ-').get_text())
       # except AttributeError:
        #    room_tip.append('Nan')


    #helpful votes
    helpful_votes = []
    for rev in all_review:
        try:
            helpful_votes.append(rev.find(class_='social-statistics-bar-SocialStatisticsBar__counts--3Zm4V').get_text())
        except AttributeError:
            helpful_votes.append('0 Helpful votes')

    #manager response
    manager_response = []
    manager_response_date = []
    for rev in all_review:
        try:
            manager_response.append(rev.find(class_='location-review-review-list-parts-OwnerResponse__containerStyles--1wv-S').get_text())


            man_date = rev.find_all(class_='location-review-review-list-parts-OwnerResponse__responseDate--4i8wE')
            for i in man_date:
                manager_response_date.append(i.get_text())
            #manager_response_date.append(rev.find_all(class_='location-review-review-list-parts-OwnerResponse__responseDate--4i8wE'))
        except AttributeError:
            manager_response.append('Nan')
            manager_response_date.append('Nan')
        
        
    df_temp = pd.DataFrame(list(zip(date_r, mem_name, rating, full_text_rev, stay_date, helpful_votes, manager_response, manager_response_date)), 
                      columns =['ReviewDate', 'ReviewerName', 'Rating', 'TextReview', 'StayDate', 'HelpfulVotes', 'ManagerResponseText', 'ManagerResponseDate']) 
    
    df_reviews = df_reviews.append(df_temp)

In [5]:
df_reviews.tail()

,ReviewDate,ReviewerName,Rating,TextReview,StayDate,HelpfulVotes,ManagerResponseText,ManagerResponseDate
0,Aug 2004,dansten,5.0,My partner and I spent two weeks in a water vi...,Nan,0 Helpful votes,Nan,Nan
1,Aug 2004,Destination15181,5.0,Greetings from Paradise....it has to be seen t...,Nan,0 Helpful votes,Nan,Nan
2,Aug 2004,Mary1967,5.0,We holidayed here from 7th-21st June 04. We we...,Nan,0 Helpful votes,Nan,Nan
3,Aug 2004,NattyPhillips,5.0,Kuredu was complete paradise on Earth. I went...,Nan,0 Helpful votes,Nan,Nan
4,Jun 2004,Adventure31736,5.0,Fantastic. Probably the best (and most gorge...,Nan,0 Helpful votes,Nan,Nan


In [4]:
df_reviews.shape

(7475, 8)

In [7]:
df_reviews.to_csv(r'KureduReviews_19_04_2020.csv', index = False)